In [44]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pickle
import random
from collections import Counter

In [45]:
lemmetizer = WordNetLemmatizer()
number_of_lines = 5332

In [46]:
# function for creating a bag of wnated words 
def create_lexicon(pos, neg):
    lexicon = []
    for file in [pos, neg]:
        with open (file, 'r') as f:
            contents = f.readlines()
            for line in contents[:number_of_lines]:
                all_words = word_tokenize(line.lower())
                lexicon += list(all_words)
    
    lexicon = [lemmetizer.lemmatize(word) for word in lexicon]
    word_counts = Counter(lexicon)
    l2 = []
    
    for word in word_counts:
        if 1000 > word_counts[word] > 50:
            l2.append(word)
    
    print(len(l2))
    return l2

In [47]:
# function for counting sentences words that are present in the bag of words
def sample_handeling(sample, lexicon, classification):
    featureset = []
    
    with open(sample, 'r') as f:
        contents = f.readlines()
        for line in contents[:number_of_lines]:
            current_words = word_tokenize(line.lower())
            current_words = [lemmetizer.lemmatize(word) for word in current_words]
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    features[index_value] += 1
                    
            features = list(features)
            featureset.append([features, classification])
            
    return featureset

In [55]:
# function for creating training and test data
def create_feature_sets_and_labels(pos, neg, test_size = 0.1):
    lexicon = create_lexicon(pos, neg)
    features = []
    features += sample_handeling(pos, lexicon, [1, 0])
    features += sample_handeling(neg, lexicon, [0, 1])
    random.shuffle(features)
    
    features = np.array(features)
    
    testing_size = int(len(features) * test_size)
    
    x_train = list(features[:-testing_size, 0])
    y_train = list(features[:-testing_size, 1])
    
    x_test = list(features[testing_size:, 0])
    y_test = list(features[testing_size:, 1])
    
    return x_train, y_train, x_test, y_test

In [ ]:
if __name__ == '__main()__':
    x_train, y_train, x_test, y_test = create_feature_sets_and_labels('pos.txt', 'neg.txt', test_size = 0.1)
    with open('sentiment.pickle', 'wb') as file:
        pickle.dump([x_train, y_train, x_test, y_test], f)